In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# train_err = pd.read_csv('./preprocessed data/new_train_err.csv', parse_dates=['time'])
train = pd.read_csv('./training set/train_case2.csv')
train_problem = pd.read_csv('./preprocessed data/new_train_problem.csv', parse_dates=['time'])
test = pd.read_csv('./training set/test.csv')
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

## quality columns 모두 제거

In [8]:
# train_quality
qual_col = list(train.iloc[:, -11:-3].columns)
train.drop(columns=qual_col, inplace=True)
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.33.1185,fwver_04.73.2571,fwver_04.82.1730,fwver_04.82.1778,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model,multiple_fwver
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.085244,-0.296006,-0.075511,-0.052361,-0.037669,0.025586,0.015363,-0.026017,-0.652306,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.071956,-0.075181,-0.149787,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.442781,0,0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,1.267385,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,0.042137,-0.075511,-0.052361,-0.037669,-0.048693,0.015363,-0.034766,0.560808,-0.279724,4.832761,2.461897,5.213885,1.147012,0.066194,0.877285,-0.590355,-0.154519,0.339732,-0.198956,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.442781,0,1
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.000473,-0.288962,-0.043544,-0.052361,-0.037669,-0.038082,0.015363,-0.052263,-0.753398,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.016696,-0.075181,-0.458185,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.272376,0,0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,-0.277099,-0.01059,-0.019935,-0.012796,0.079299,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.281917,-0.075511,-0.052361,-0.037669,-0.048693,-0.064734,-0.069761,-0.854491,-0.188357,-0.225162,-0.114028,-0.252677,-0.174717,-0.071956,-0.075181,0.951632,-0.154519,-0.194419,-0.198956,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.442781,0,1
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.011326,-0.00921,-0.084309,-0.008165,-0.009902,-0.035379,0.164038,-0.01059,-0.019935,-0.012796,-0.250725,-0.026621,-0.008165,-0.041437,-0.008165,-0.008165,-0.016768,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.296006,-0.011578,-0.052361,-0.037669,-0.048693,0.175556,-0.047889,0.560808,-0.279724,-0.225162,-0.114028,-0.252677,0.028626,-0.071956,-0.075181,-0.105730,-0.154519,0.126072,-0.198956,0.364376,-0.061546,-0

In [13]:
# test_quality
qual_col = list(test.iloc[:, -11:-3].columns)
test.drop(columns=qual_col, inplace=True)
test

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.33.1185,fwver_04.73.2571,fwver_04.82.1730,fwver_04.82.1778,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,period,multiple_model,multiple_fwver
0,0.432273,-0.282718,1.465372,-0.138438,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,4.880865,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,0.023916,-0.076512,-0.047344,-0.056717,-0.022843,-0.144765,-0.045624,-0.563053,0.217326,2.989399,-0.08513,1.096973,0.269713,1.0,1.0
1,-0.191518,-0.282718,-0.227667,-0.190118,0.051198,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,0.051412,-0.228085,0.006710,-0.047344,-0.056717,-0.022843,0.676954,-0.036905,-0.752183,-0.121844,-0.202841,-0.08513,-0.190181,0.095698,0.0,0.0
2,-0.033722,0.638161,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,0.297034,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,0.005477,-0.048771,-0.047344,-0.056717,-0.022843,-0.144765,-0.051436,0.993630,-0.121844,-0.202841,-0.08513,-0.190181,0.443729,0.0,1.0
3,-0.180302,0.051777,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,0.092647,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,0.202160,-0.048771,-0.047344,-0.056717,-0.022843,0.060665,-0.033999,-1.406862,-0.121844,0.861239,-0.08513,1.096973,0.095698,0.0,1.0
4,-0.041880,-0.282718,0.166889,0.017435,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,0.085206,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,-0.209646,-0.076512,-0.047344,-0.056717,-0.022843,0.060665,-0.025281,-0.373923,0.104270,-0.202841,-0.08513,-0.190181,0.443729,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,0.020321,-0.282718,0.479616,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,-0.267318,-0.013322,-0.012829,-0.014172,-0.158979,-0.017038,-0.01172,-0.0421,-0.008166,-0.0083,-0.009566,-0.008195,-0.022131,-0.009703,-0.195694,-0.031262,-0.053793,-0.172767,0.034451,-0.047344,-0.056717,-0.022843,0.471524,-0.028187,0.266208,0.330383,-0.202841,-0.08513,-0.190181,0.443729,0.0,0.0
14994,-0.132632,0.242479,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,-0.008181,-0.012279,-0.008477,-0.011854,-0.129828,-0.008226,-0.013014,-0.03182,1.178642,-0.013322,-0.012829,-